DESAFIO 300 - CALCULAR DISTÂNCIA MAIS PRÓXIMA DE UMA UNIDADE DE SAÚDE AO ACIDENTE.

In [3]:
import pandas as pd
import math

In [4]:
def calcular_distancia(lat1, lon1, lat2, lon2):
    # Raio médio da Terra (em Km)
    raio_terra = 6371.0
    
    # Convertendo as latitudes e longitudes para radianos
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    
    # Diferença das latitudes e longitudes
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    # Fórmula Harservine
    a = math.sin(dlat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    
    # Distância em quilômetros
    distancia = raio_terra * c
    
    return distancia
    

In [5]:
data_acidentes_filtrados = pd.read_csv('data_acidentes_filtrado.csv',sep=',')
data_acidentes_filtrados.columns

Index(['id', 'pesid', 'data_inversa', 'dia_semana', 'horario', 'uf', 'br',
       'km', 'municipio', 'causa_principal', 'causa_acidente',
       'ordem_tipo_acidente', 'tipo_acidente', 'classificacao_acidente',
       'fase_dia', 'sentido_via', 'condicao_metereologica', 'tipo_pista',
       'tracado_via', 'uso_solo', 'id_veiculo', 'tipo_veiculo',
       'tipo_envolvido', 'estado_fisico', 'idade', 'sexo', 'ilesos',
       'feridos_leves', 'feridos_graves', 'mortos', 'latitude', 'longitude'],
      dtype='object')

In [6]:
data_unidades = pd.read_csv('data_unidades.csv',sep=',')
data_unidades.columns

Index(['codigo_cnes', 'nome_razao_social', 'nome_fantasia', 'tipo_gestao',
       'codigo_tipo_unidade', 'codigo_cep_estabelecimento',
       'endereco_estabelecimento', 'numero_estabelecimento',
       'bairro_estabelecimento', 'latitude_estabelecimento_decimo_grau',
       'longitude_estabelecimento_decimo_grau',
       'codigo_identificador_turno_atendimento', 'descricao_turno_atendimento',
       'estabelecimento_faz_atendimento_ambulatorial_sus',
       'codigo_estabelecimento_saude', 'codigo_uf', 'codigo_municipio',
       'descricao_natureza_juridica_estabelecimento',
       'estabelecimento_possui_centro_cirurgico',
       'estabelecimento_possui_centro_obstetrico',
       'estabelecimento_possui_centro_neonatal',
       'estabelecimento_possui_atendimento_hospitalar',
       'estabelecimento_possui_servico_apoio',
       'estabelecimento_possui_atendimento_ambulatorial',
       'codigo_atividade_ensino_unidade', 'descricao_tipo_unidade', 'CodMun',
       'UF', 'Cod UF', 'munic

**VAMOS AGRUPAR AS LINHAS POR ID (ACIDENTE) E MUNICIPIO E PEGAR APENAS AS LATITUDES E LONGITUTES PARA OTIMIZAR O PROCESSAMENTO**

In [7]:
# Função para substituir vírgulas por pontos e converter para float
def parse_float(value):
    return float(value.replace(',', '.'))

# Converter colunas de latitude e longitude para float
data_acidentes_filtrados['latitude'] = data_acidentes_filtrados['latitude'].apply(parse_float)
data_acidentes_filtrados['longitude'] = data_acidentes_filtrados['longitude'].apply(parse_float)


# Agrupar por ID e município, e selecionar as colunas de latitude e longitude
grupo_por_id_municipio = data_acidentes_filtrados.groupby(['id', 'municipio','uf'])[['latitude', 'longitude']].mean()


# Resetar o índice para que 'id' e 'municipio' se tornem colunas novamente
grupo_por_id_municipio= grupo_por_id_municipio.reset_index()


grupo_por_id_municipio.head()

,id,municipio,uf,latitude,longitude
0,496506,MANHUACU,MG,-20.241739,-42.158680
1,496507,LAVRAS,MG,-21.179290,-45.122566
2,496512,ESTRELA,RS,-29.501601,-51.947844
3,496513,NOSSA SENHORA DO SOCORRO,SE,-10.903851,-37.128677
4,496515,SAO JOSE DOS PINHAIS,PR,-25.584265,-49.176188


In [12]:
# Criar listas para armazenar os valores
distancias_minimas = []
codigo_cnes_unidades = []
lat_unidades = []
lon_unidades = []

# Iterar sobre as linhas em 'grupo_por_id_municipio'
for index_acidente, row_acidente in grupo_por_id_municipio.iterrows():
    lat_acidente = row_acidente['latitude']
    lon_acidente = row_acidente['longitude']
    municipio_acidente = row_acidente[['municipio','uf']]  
    
    distancia_minima = float('inf')  # Inicializar com um valor infinito
    codigo_cnes_minima = None
    lat_unidade_minima = None
    lon_unidade_minima = None
    
    # Filtrar 'data_unidades' para selecionar apenas as linhas com o mesmo município e UF
    unidades_mesmo_municipio = data_unidades[
        (data_unidades['municipio'] == municipio_acidente['municipio']) &
        (data_unidades['UF'] == municipio_acidente['uf'])]
    
    # Iterar sobre as linhas em 'unidades_mesmo_municipio'
    for index_unidade, row_unidade in unidades_mesmo_municipio.iterrows():
        lat_unidade = row_unidade['latitude_estabelecimento_decimo_grau']
        lon_unidade = row_unidade['longitude_estabelecimento_decimo_grau']
        
        # Calcular a distância usando sua função 'calcular_distancia'
        distancia = calcular_distancia(lat_acidente, lon_acidente, lat_unidade, lon_unidade)
        
        # Atualizar a distância mínima e os valores da unidade se a distância calculada for menor
        if distancia < distancia_minima:
            distancia_minima = distancia
            codigo_cnes_minima = row_unidade['codigo_cnes']  # Supondo que essa coluna exista
            lat_unidade_minima = lat_unidade
            lon_unidade_minima = lon_unidade
    
    # Adicionar os valores às listas
    distancias_minimas.append(distancia_minima)
    codigo_cnes_unidades.append(codigo_cnes_minima)
    lat_unidades.append(lat_unidade_minima)
    lon_unidades.append(lon_unidade_minima)

# Adicionar as listas como novas colunas em 'grupo_por_id_municipio'
grupo_por_id_municipio['distancia_minima'] = distancias_minimas
grupo_por_id_municipio['codigo_cnes'] = codigo_cnes_unidades
grupo_por_id_municipio['lat_unidade_minima'] = lat_unidades
grupo_por_id_municipio['lon_unidade_minima'] = lon_unidades

**VAMOS AGRUPAR TODAS AS INFORMAÇÕES PARA GERAR O DATASET FINAL**

In [13]:
# Excluir as colunas 'municipio', 'latitude' e 'longitude'
colunas_para_excluir = ['municipio', 'latitude', 'longitude']
grupo_por_id_municipio.drop(columns=colunas_para_excluir, inplace=True)

In [14]:
#Realizar o merge usando a coluna 'id' como chave de junção
data_acidentes_distancias = data_acidentes_filtrados.merge(grupo_por_id_municipio, on='id', how='left')
data_acidentes_distancias.head(2)

,id,pesid,data_inversa,dia_semana,horario,uf_x,br,km,municipio,causa_principal,...,feridos_leves,feridos_graves,mortos,latitude,longitude,uf_y,distancia_minima,codigo_cnes,lat_unidade_minima,lon_unidade_minima
0,496506,1082142.0,2023-01-01,domingo,00:15:00,MG,116.0,587,MANHUACU,Sim,...,1,0,0,-20.241739,-42.15868,MG,13.132012,44628,-20.258,-42.034
1,496506,1082142.0,2023-01-01,domingo,00:15:00,MG,116.0,587,MANHUACU,Não,...,1,0,0,-20.241739,-42.15868,MG,13.132012,44628,-20.258,-42.034


In [15]:
# Realizar o merge usando a coluna 'codigo_cnes' como chave de junção
data_acidentes_unidades_merged = data_acidentes_distancias.merge(data_unidades, on='codigo_cnes', how='left')
data_acidentes_unidades_merged.head(2)

,id,pesid,data_inversa,dia_semana,horario,uf_x,br,km,municipio_x,causa_principal,...,estabelecimento_possui_centro_neonatal,estabelecimento_possui_atendimento_hospitalar,estabelecimento_possui_servico_apoio,estabelecimento_possui_atendimento_ambulatorial,codigo_atividade_ensino_unidade,descricao_tipo_unidade,CodMun,UF,Cod UF,municipio_y
0,496506,1082142.0,2023-01-01,domingo,00:15:00,MG,116.0,587,MANHUACU,Sim,...,0.0,0.0,0,0,4,CONSULTORIO ISOLADO,3139409,MG,31,MANHUACU
1,496506,1082142.0,2023-01-01,domingo,00:15:00,MG,116.0,587,MANHUACU,Não,...,0.0,0.0,0,0,4,CONSULTORIO ISOLADO,3139409,MG,31,MANHUACU


In [16]:
data_acidentes_unidades_merged.to_csv('data_acidentes_unidades.csv', index = False)

In [20]:
calcular_distancia(-8.3791974,-55.11727824, -3.203000,-52.206000)

659.4934281648815